In [1]:
from src.arguments import ModelArguments, DataArguments
from src.model.model import MMEBModel
from src.model.processor import load_processor, LLAVA_QWEN2, VLM_IMAGE_TOKENS, LlavaQwen2_process_fn 
from src.utils.basic_utils import batch_to_device
from PIL import Image
import torch

DropoutAddRMSNorm of flash_attn is not installed!!!


/venv/main/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/workspace/VLM2Vec/src/model/baseline_backbone/internvideo2/modeling_internvideo2.py:539: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


In [2]:
model_args = ModelArguments(
    model_name='apple/FastVLM-0.5B',
    pooling='last',
    normalize=True,
    model_backbone='llava_qwen2',
    # lora=True
)
data_args = DataArguments()
processor = load_processor(model_args, data_args)
model = MMEBModel.load(model_args)
model = model.to('cuda', dtype=torch.bfloat16)
model.eval()

[2025-11-28 10:21:44,153] INFO [src.utils.basic_utils:21] Loading processor from: apple/FastVLM-0.5B
[2025-11-28 10:21:44,154] INFO [src.utils.basic_utils:21] Customized: load LlavaQwen2Processor
[2025-11-28 10:21:44,160] DEBUG [urllib3.connectionpool:1049] Starting new HTTPS connection (1): huggingface.co:443
[2025-11-28 10:21:44,477] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
[2025-11-28 10:21:44,488] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/tokenizer_config.json HTTP/1.1" 200 0
[2025-11-28 10:21:45,036] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/config.json HTTP/1.1" 307 0
[2025-11-28 10:21:45,054] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d

MMEBModel(
  (encoder): LlavaQwen2ForCausalLM(
    (model): LlavaQwen2Model(
      (embed_tokens): Embedding(151936, 896)
      (layers): ModuleList(
        (0-23): 24 x Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        )
      )
   

In [9]:
inputs = processor(text=f'{VLM_IMAGE_TOKENS[LLAVA_QWEN2]} Represent the given image with the following question: What is in the image',
                   images=Image.open('assets/example.jpg'),
                   return_tensors="pt")
inputs = {key: value.to('cuda') for key, value in inputs.items()}
print(inputs['input_ids'].shape)
print(inputs['images'].shape)

torch.Size([1, 15])
torch.Size([1, 3, 1024, 1024])


In [11]:
with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
    image_features = model.encoder.encode_images(inputs['images'])
print(image_features.shape)

torch.Size([1, 256, 896])


In [ ]:
vision_tower = model.encoder.get_vision_tower()


In [17]:
# xem toàn bộ config trong model
print(vision_tower.__dict__)


{'training': False, '_parameters': {}, '_buffers': {}, '_non_persistent_buffers_set': set(), '_backward_pre_hooks': OrderedDict(), '_backward_hooks': OrderedDict(), '_is_full_backward_hook': None, '_forward_hooks': OrderedDict(), '_forward_hooks_with_kwargs': OrderedDict(), '_forward_hooks_always_called': OrderedDict(), '_forward_pre_hooks': OrderedDict(), '_forward_pre_hooks_with_kwargs': OrderedDict(), '_state_dict_hooks': OrderedDict(), '_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_post_hooks': OrderedDict(), '_modules': {'vision_tower': MCi(
  (model): FastViT(
    (patch_embed): Sequential(
      (0): MobileOneBlock(
        (se): Identity()
        (activation): GELU(approximate='none')
        (reparam_conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
      (1): MobileOneBlock(
        (se): Identity()
        (activation): GELU(approximate='none')
        (reparam_conv): Conv2d(96, 96, kernel

In [22]:
vision_tower.cfg_only['image_cfg']['patch_size']

64

In [1]:
from src.arguments import ModelArguments, DataArguments
from src.model.model import MMEBModel
from src.model.processor import load_processor, QWEN2_VL, VLM_IMAGE_TOKENS, Qwen2_VL_process_fn
from src.utils.basic_utils import batch_to_device
from PIL import Image
import torch

model_args = ModelArguments(
    model_name='Qwen/Qwen2-VL-7B-Instruct',
    checkpoint_path='TIGER-Lab/VLM2Vec-Qwen2VL-7B',
    pooling='last',
    normalize=True,
    model_backbone='qwen2_vl',
    lora=True
)
data_args = DataArguments()

processor = load_processor(model_args, data_args)
model = MMEBModel.load(model_args)
model = model.to('cuda', dtype=torch.bfloat16)
model.eval()

# Image + Text -> Text
inputs = processor(text=f'{VLM_IMAGE_TOKENS[QWEN2_VL]} Represent the given image with the following question: What is in the image',
                   images=Image.open('../../../assets/example.jpg'),
                   return_tensors="pt")
inputs = {key: value.to('cuda') for key, value in inputs.items()}
inputs.keys()

/venv/main/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/workspace/VLM2Vec/src/model/baseline_backbone/internvideo2/modeling_internvideo2.py:539: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
[2025-11-28 11:06:58,913] INFO [src.utils.basic_utils:21] Loading processor from: TIGER-Lab/VLM2Vec-Qwen2VL-7B
[2025-11-28 11:06:58,916] DEBUG [urllib3.connectionpool:1049] Starting new HTTPS connection (1): huggingface.co:443


DropoutAddRMSNorm of flash_attn is not installed!!!


[2025-11-28 11:06:59,306] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /TIGER-Lab/VLM2Vec-Qwen2VL-7B/resolve/main/preprocessor_config.json HTTP/1.1" 307 0
[2025-11-28 11:06:59,565] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/TIGER-Lab/VLM2Vec-Qwen2VL-7B/5aec77d4014be33b00aff8f7856865214682f3d9/preprocessor_config.json HTTP/1.1" 200 0
[2025-11-28 11:06:59,569] DEBUG [filelock:331] Attempting to acquire lock 139986966499232 on /workspace/.hf_home/hub/.locks/models--TIGER-Lab--VLM2Vec-Qwen2VL-7B/2ef5e95d7d22f4b301fbf6bd6de694f5918a33fd.lock
[2025-11-28 11:06:59,572] DEBUG [filelock:334] Lock 139986966499232 acquired on /workspace/.hf_home/hub/.locks/models--TIGER-Lab--VLM2Vec-Qwen2VL-7B/2ef5e95d7d22f4b301fbf6bd6de694f5918a33fd.lock
[2025-11-28 11:06:59,837] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "GET /api/resolve-cache/models/TIGER-Lab/VLM2Vec-Qwen2VL-7B/5aec77d4014be33b00aff8f7856865214682f3d9

preprocessor_config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

[2025-11-28 11:06:59,864] DEBUG [filelock:364] Attempting to release lock 139986966499232 on /workspace/.hf_home/hub/.locks/models--TIGER-Lab--VLM2Vec-Qwen2VL-7B/2ef5e95d7d22f4b301fbf6bd6de694f5918a33fd.lock
[2025-11-28 11:06:59,865] DEBUG [filelock:367] Lock 139986966499232 released on /workspace/.hf_home/hub/.locks/models--TIGER-Lab--VLM2Vec-Qwen2VL-7B/2ef5e95d7d22f4b301fbf6bd6de694f5918a33fd.lock
[2025-11-28 11:07:00,192] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /TIGER-Lab/VLM2Vec-Qwen2VL-7B/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
[2025-11-28 11:07:00,452] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/TIGER-Lab/VLM2Vec-Qwen2VL-7B/5aec77d4014be33b00aff8f7856865214682f3d9/tokenizer_config.json HTTP/1.1" 200 0
[2025-11-28 11:07:00,455] DEBUG [filelock:331] Attempting to acquire lock 139986966500864 on /workspace/.hf_home/hub/.locks/models--TIGER-Lab--VLM2Vec-Qwen2VL-7B/2602c8d951d36405801564bcbafe8ea0

...

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

[2025-11-28 11:08:44,411] DEBUG [filelock:364] Attempting to release lock 139986967002320 on /workspace/.hf_home/hub/.locks/models--Qwen--Qwen2-VL-7B-Instruct/eab4f4dc1abf860794c98ce3759b4ace1059fc0fc041ede76b88988b3557c132.lock
[2025-11-28 11:08:44,413] DEBUG [filelock:367] Lock 139986967002320 released on /workspace/.hf_home/hub/.locks/models--Qwen--Qwen2-VL-7B-Instruct/eab4f4dc1abf860794c98ce3759b4ace1059fc0fc041ede76b88988b3557c132.lock
[2025-11-28 11:08:44,701] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /Qwen/Qwen2-VL-7B-Instruct/resolve/main/model-00002-of-00005.safetensors HTTP/1.1" 302 0
[2025-11-28 11:08:44,706] DEBUG [filelock:331] Attempting to acquire lock 139986966997088 on /workspace/.hf_home/hub/.locks/models--Qwen--Qwen2-VL-7B-Instruct/0546b7cd070ff239de0f61a28770de6a0ac517ad313071e151653e38911ffb25.lock
[2025-11-28 11:08:44,708] DEBUG [filelock:334] Lock 139986966997088 acquired on /workspace/.hf_home/hub/.locks/models--Qwen--Qwen2-VL-7B-Instru

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.
[2025-11-28 11:08:51,437] DEBUG [filelock:364] Attempting to release lock 139986966997088 on /workspace/.hf_home/hub/.locks/models--Qwen--Qwen2-VL-7B-Instruct/0546b7cd070ff239de0f61a28770de6a0ac517ad313071e151653e38911ffb25.lock
[2025-11-28 11:08:51,438] DEBUG [filelock:367] Lock 139986966997088 released on /workspace/.hf_home/hub/.locks/models--Qwen--Qwen2-VL-7B-Instruct/0546b7cd070ff239de0f61a28770de6a0ac517ad313071e151653e38911ffb25.lock


KeyboardInterrupt: 